# Mirai ALM Tool

### General Resources:
- Hull, OPTIONS, FUTURES, AND OTHER DERIVATIVES
- Hilpisch, Derivatives Analytics with Python

## 1.1 Sensitivity

Requirement:
- Imagine a portfolio of securities whose value can be approximately represented by a single risk factor x (random variable), as follows: ppv=f(x) = k1 * e^(-T1*x),
where k1 and T1 are constants. 

- The portfolio owner wants to add a new security to the portfolio, which has the generic form g(x, k2, T2), where x is the same
random variable. 

- The constants k1 and k2 are the nominal values of the securities. 

- Create a function that returns four values: 
    - the value of the old portfolio, the
value of the new portfolio, the sensitivity of the old portfolio to factor x, and the sensitivity of the new portfolio to factor x taking into account the weighted face
value (positive for long positions, negative for short positions). The inputs to the function should be k1, T1, x and g(x, k2, T2).

### Resources: 
#### - https://www.albany.edu/~bd445/Economics_466_Financial_Economics_Slides_Spring_2014/Duration.pdf

In [1]:
import numpy as np 
import  pandas as pd

def portfolio_values_and_sensitivities(k1, T1, x, g_func, k2, T2):
    # value of the old portfolio
    value_stock1 = k1 * np.exp(-T1 * x)
    # sensitivity of the stock to factor x - As derivatives of value stocks
    sensitivity_stock_1 = -k1 * T1 * np.exp(-T1 * x)
    # value of the 2 stock portfolio using the generic function g(x, k2, T2)
    value_stock2 = g_func(x, k2, T2)
    # sensitivity of stock 2 to factor x 
    sensitivity_stock2 = -g_func(x, k2, T2) * T2
    # Portfolio as sum of stock and because the Portfolio has the sensitivity to factor x is a sum
    portfolio_value_old = k1 + k2
    portfolio_value_new = value_stock1 + value_stock2 
    portfolio_sensitivity = sensitivity_stock_1 + sensitivity_stock2 
    
    return value_stock1, value_stock2, sensitivity_stock_1, sensitivity_stock2 , portfolio_value_old ,portfolio_value_new  , portfolio_sensitivity

# Example of the generic function g(x, k2, T2) where g(x, k2, T2) = k2 * e^(-T2*x)
def generic_security_value(x, k2, T2):
    return k2 * np.exp(-T2 * x)

In [2]:
# Example usage
np.random.seed(22)
x = np.random.choice(24)/10
k1 = 1000
T1 = 0.1
k2 = 800
T2 = 0.05

value_stock1, value_stock2, sensitivity_stock_1, sensitivity_stock2 , portfolio_value_old ,portfolio_value_new  , portfolio_sensitivity = portfolio_values_and_sensitivities(k1, T1, x, generic_security_value, k2, T2)

print("Value of the stock1:", value_stock1)
print("Value of the stock2:", value_stock2)
print("Sensitivity of the stock1  to factor x:", sensitivity_stock_1)
print("Sensitivity of the stock1 to factor x:", sensitivity_stock2)
print("Value of the old portfolio with the 2 stock:", portfolio_value_old)
print("Value of the portfolio with the 2 stock:", portfolio_value_new)
print("Sensitivity of the portfolio with the 2 stock:", portfolio_sensitivity)

Value of the stock1: 810.5842459701871
Value of the stock2: 720.2596180690125
Sensitivity of the stock1  to factor x: -81.05842459701871
Sensitivity of the stock1 to factor x: -36.012980903450625
Value of the old portfolio with the 2 stock: 1800
Value of the portfolio with the 2 stock: 1530.8438640391996
Sensitivity of the portfolio with the 2 stock: -117.07140550046934


## 1.2 Stochastic analysis

Imagine that x is a random variable that follows a simple stochastic process;
- the Wiener process: dx = x * q * dW
- Build a function that calculates the value of the new portfolio
- taking as inputs: 
    - k1
    - T1
    - g(x, k2, T2)
    - q 
    - deltaStepOfWiener 
    - numMaxFinalSteps

### info about winer process / geometric brownian motion 
##### - https://hpaulkeeler.com/wiener-or-brownian-motion-process/
##### - https://pyquantnews.com/how-to-simulate-stock-prices-with-python/

In [3]:
def winer_process(x , q , dw):
    dx = x * q * dw 
    return dx

In [4]:
import numpy as np
def simulate(k1, T1, generic_security_value, q, k2, T2, delta_step_of_wiener, num_max_final_steps):
    # Start at 0
    value_new_portfolio = 0
    sensitivity = 0
    x = 0
    for i in range(num_max_final_steps):
        delta_W = np.random.normal() * np.sqrt(delta_step_of_wiener)
        x = winer_process(k1, q, delta_W)
        # assumption: g_func = S0 , mean and variance are 0 , 1 as X is Random from the normal distribution.
        value_new_portfolio += generic_security_value(x, k2, T2) * np.exp(-T1 * delta_step_of_wiener)
        sensitivity += -generic_security_value(x, k2, T2) * T2 * np.exp(-T1 * delta_step_of_wiener)
    return value_new_portfolio, sensitivity

In [5]:
# Example usage
k1 = 100
T1 = 1
k2 = 800
T2 = 5
q = 0.2
delta_step_of_wiener = 0.01
num_max_final_steps = 10000

value_new_portfolio , sensy= simulate(k1, T1,  generic_security_value, q,k2 , T2, delta_step_of_wiener, num_max_final_steps )

print("Value of the new portfolio considering the Wiener process:", value_new_portfolio)

Value of the new portfolio considering the Wiener process: 4.313253795103076e+19


## 1.3 Optimization

Imagine the portfolio owner wants to offset the risk associated with x in the new portfolio. 
- To achieve this, the T2 parameter, instead of being a constant, can be adjusted. 
- Develop a routine that requires inputs such as k1, T1, x, and g(x, k2, t2) to identify
the potential values of t2 that offset all risk associated with x.
- Use any optimization techniques as needed.

####  Resources :
##### - https://realpython.com/python-scipy-cluster-optimize/ (Optimize Module in SciPy)
##### - https://www.investopedia.com/terms/o/offsetting-transaction.asp#:~:text=Key%20Takeaways,as%20close%20as%20possible)%20instrument.

In [6]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from scipy.optimize import minimize_scalar

In [7]:
def find_offsetting_T2(k1, T1, g_func, q, delta_step_of_wiener, num_max_final_steps):
    # Define the objective function to trace the function (lambda) and minimize the sensitivity_new_portfolio (simulate function) with T2 as value to be optimazide for find the minimum of the function
    objective_function = lambda T2: simulate(k1, T1, g_func, q, k2, T2, delta_step_of_wiener, num_max_final_steps)[1]
    
    # Perform the optimization to find the offsetting T2 value
    result = minimize_scalar(objective_function)
    
    return result , result.x

In [8]:
# Example usage
k1 = 1000
T1 = 0.1
q = 0.2
delta_step_of_wiener = 0.01
num_max_final_steps = 1000
x = 1.0
k2 = 800
info , T2_offsetting = find_offsetting_T2(k1, T1,generic_security_value, q, delta_step_of_wiener, num_max_final_steps)

print("info of the T2 optimization:", info)
print("value T2 that Offset, value:", T2_offsetting)

info of the T2 optimization:      fun: -inf
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.48e-08 )'
    nfev: 43
     nit: 35
 success: True
       x: 27.416408113280472
value T2 that Offset, value: 27.416408113280472
